In [ ]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])

In [ ]:
 train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
 test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

 train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
 test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
image, label = train_data[0]

In [ ]:
class_name = ['plane', 'car', 'cat', 'bird', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
class NuralNet(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(3, 12, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 =nn.Conv2d(12, 24, 5)
    self.fc1 = nn.Linear(24 *5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
net = NuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(10):
  print(f'Training epoch {epoch}')
  running_loss = 0.0

  for i, data in enumerate(train_loader):
    inputs, labels = data

    optimizer.zero_grad()

    outputs = net(inputs)

    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print(f'loss: {running_loss / len(train_loader):.4f}')

Training epoch 0
loss: 1.6144
Training epoch 1
loss: 1.4546
Training epoch 2
loss: 1.3520
Training epoch 3
loss: 1.2710
Training epoch 4
loss: 1.1965
Training epoch 5
loss: 1.1317
Training epoch 6
loss: 1.0756
Training epoch 7
loss: 1.0272
Training epoch 8
loss: 0.9855
Training epoch 9
loss: 0.9474


In [ ]:
torch.save(net.state_dict(), 'trained_net.pth')

In [ ]:
net = NuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<ipython-input-35-eebb95e73455>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_net.pth'))


<All keys matched successfully>

In [ ]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
  for data in test_loader:
    inputs, labels = data
    outputs = net(inputs)
    _, predicted =torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  accuracy = 100 + correct / total

  print(accuracy)

100.6401


In [ ]:
new_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
])

def load_image(image_path):
  image = Image.open(image_path)
  image = new_transform(image)
  image = image.unsqueeze(0)
  return image

image_paths = ['1.jpg', '2.jpg']
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
  for image in images:
    output = net(image)
    _, predicted =torch.max(output, 1)
    print(class_name[predicted.item()])

plane
dog
